In [1]:
! pip install transformers[sentencepiece] -Uqq

In [2]:
import json, pdb
from functools import partial
from pathlib import Path
from pprint import pprint

from datasets import concatenate_datasets, load_dataset, Dataset, DatasetDict

In [73]:
import torch

In [74]:
torch.__version__

'1.8.1'

In [75]:
print(torch.version.cuda)

None


In [3]:
mrpc = load_dataset("glue", "mrpc")
mrpc

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Reusing dataset glue (C:\Users\alex\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
training = mrpc["train"]
training

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [5]:
print(training.info.description)

GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.




In [6]:
print(training[0])

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


In [7]:
print(training[3])

{'sentence1': 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'sentence2': 'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .', 'label': 0, 'idx': 3}


In [8]:
print(training.features)

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}


In [9]:
#Loading data to csv file
mrpc["train"].to_csv("mrpc_train.csv")
mrpc["validation"].to_csv("mrpc_val.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

104594

In [10]:
raw_data = load_dataset("csv", data_files="mrpc_train.csv")
raw_data

Using custom data configuration default-4c96cc6de183a8ee


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\alex\.cache\huggingface\datasets\csv\default-4c96cc6de183a8ee\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
})

In [11]:
#JSON files
mrpc["train"].to_json("mrpc_train.json", lines=False, orient="table")
mrpc["validation"].to_json("mrpc_val.json", lines=False, orient="table")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

124608

In [12]:
with Path("mrpc_train.json").open() as f:
    mrpc_train_json = json.load(f)
    
mrpc_train_json.keys()

dict_keys(['schema', 'data'])

In [13]:
mrpc_train_json["data"][:3]

[{'index': 0,
  'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  'label': 1,
  'idx': 0},
 {'index': 1,
  'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  'label': 0,
  'idx': 1},
 {'index': 2,
  'sentence1': 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  'sentence2': "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
  'label': 1,
  'idx': 2}]

In [14]:
raw_data = load_dataset("json", data_files="mrpc_train.json", field="data")

Using custom data configuration default-bb00dca07e992c07


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to C:\Users\alex\.cache\huggingface\datasets\json\default-bb00dca07e992c07\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['index', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
})

In [16]:
#jsonl (json lines)
mrpc["train"].to_json("mrpc_train.jsonl")
mrpc["validation"].to_json("mrpc_val.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

119566

In [17]:
with Path("mrpc_train.jsonl").open() as f:
    raw_jsonlines = f.readlines()
    
pprint(raw_jsonlines[:3])

['{"sentence1":"Amrozi accused his brother , whom he called \\" the witness '
 '\\" , of deliberately distorting his evidence .","sentence2":"Referring to '
 'him as only \\" the witness \\" , Amrozi accused his brother of deliberately '
 'distorting his evidence .","label":1,"idx":0}\n',
 '{"sentence1":"Yucaipa owned Dominick \'s before selling the chain to Safeway '
 'in 1998 for $ 2.5 billion .","sentence2":"Yucaipa bought Dominick \'s in '
 '1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 '
 '.","label":0,"idx":1}\n',
 '{"sentence1":"They had published an advertisement on the Internet on June 10 '
 ', offering the cargo for sale , he added .","sentence2":"On June 10 , the '
 "ship 's owners had published an advertisement on the Internet , offering the "
 'explosives for sale .","label":1,"idx":2}\n']


In [18]:
raw_jsonl_datasets = load_dataset("json", data_files={"train": "mrpc_train.jsonl", "val": "mrpc_val.jsonl"})
raw_jsonl_datasets

Using custom data configuration default-8de0e48ef66be0d8


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to C:\Users\alex\.cache\huggingface\datasets\json\default-8de0e48ef66be0d8\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    val: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
})

In [19]:
#pandas
train_df = mrpc["train"].to_pandas()
val_df = mrpc["validation"].to_pandas()

train_df.head(3)

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2


In [20]:
train_set = Dataset.from_pandas(train_df)
val_set = Dataset.from_pandas(val_df)

In [21]:
train_set

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [22]:
data_dict = DatasetDict({"train": train_set, "val": val_set}) 
data_dict

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    val: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
})

In [23]:
# dataframes<->dataframes
mrpc.set_format("pandas")
type(mrpc)

datasets.dataset_dict.DatasetDict

In [24]:
#when we pull three examples it acts like a df even though we didnt change it.
mrpc["train"][:3].head()

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2


In [25]:
#we can change how many counts of each label
mrpc["train"][:]["label"].value_counts()

1    2474
0    1194
Name: label, dtype: int64

In [26]:
mrpc["validation"][:]["label"].value_counts()

1    279
0    129
Name: label, dtype: int64

In [27]:
#we can add new column(s)
train_df = mrpc["train"].to_pandas()
type(train_df)

pandas.core.frame.DataFrame

In [28]:
train_df["new_column"] = "New Column"

In [29]:
exp_train_ds = Dataset.from_pandas(train_df)
exp_train_ds

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'new_column'],
    num_rows: 3668
})

In [30]:
#we can reset formats too
mrpc.reset_format()

In [31]:
from fastai.data.all import *
from fastai.learner import *
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import Adam, OptimWrapper, params
from fastai.metrics import accuracy, F1Score
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForSequenceClassification

from blurr.data.core import *
from blurr.modeling.core import *
from blurr.utils import BLURR

C:\Users\alex\Anaconda3\envs\fastai\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\alex\Anaconda3\envs\fastai\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [32]:
#experimentation subset of data
exp_train_ds = load_dataset("imdb", split="train")
exp_train_ds

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to C:\Users\alex\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [35]:
raw_datasets = exp_train_ds.train_test_split(test_size=0.2, seed=42)
raw_datasets

Loading cached split indices for dataset at C:\Users\alex\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-99c1a29c3ffbdc47.arrow and C:\Users\alex\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-91c6eac046c0c564.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [36]:
#you can rename the 'test' set to validation if that's what you are used to (like Wayde does)
raw_datasets["validation"] = raw_datasets.pop("test")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [37]:
def is_valid(examples, is_valid=False):
    return{"is valid": is_valid}

def is_valid_batch(examples, is_valid=False):
    return{"is_valid": [is_valid for txt in examples["text"]]}

In [40]:
#we can add these is_valid checkers as features in the actual dataset
raw_datasets["train"] = raw_datasets["train"].map(partial(is_valid_batch, is_valid=False), batched=True)
raw_datasets["validation"] = raw_datasets["validation"].map(partial(is_valid_batch, is_valid=True), batched=True)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [44]:
print(raw_datasets)
print(raw_datasets["train"][1])

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'is_valid'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label', 'is_valid'],
        num_rows: 5000
    })
})
{'text': "'The Rookie' was a wonderful movie about the second chances life holds for us and also puts an emotional thought over the audience, making them realize that your dreams can come true. If you loved 'Remember the Titans', 'The Rookie' is the movie for you!! It's the feel good movie of the year and it is the perfect movie for all ages. 'The Rookie' hits a major home run!", 'label': 1, 'is_valid': False}


In [46]:
#we can select samples of our larger datasets to construct a smaller set for experimentation
train_sample = raw_datasets["train"].shuffle().select(range(1000))
val_sample = raw_datasets["validation"].shuffle().select(range(1000))

In [47]:
process_sample = concatenate_datasets([train_sample, val_sample])
process_sample

Dataset({
    features: ['text', 'label', 'is_valid'],
    num_rows: 2000
})

In [50]:
process_sample[4]

{'text': 'This must be accompanied by a special rating and warning: NOT RECOMMENDED TO NORMAL PEOPLE.<br /><br />The obsession of Daneliuc with the most dirty body functions becomes here a real nightmare. Also, it\'s evident that the man is a misanthrope, he hates everybody - his country his people, his actors, his job. And this hatred makes him blind and he forgets anymore the profession he knew long ago.<br /><br />This so called "film" is just a hideous string of disgusting images, with no artistic value and no professionist knowledge. It is an insult to good taste and to good sense. Shame, shame, shame!',
 'label': 0,
 'is_valid': False}

In [52]:
#check what our labels are
labels = process_sample.features["label"].names
labels

['neg', 'pos']

In [57]:
model = AutoModelForSequenceClassification

pretrained_model_name = "roberta-base" #bert-base-multilingual
num_labels = len(labels)

#HF objects
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(
    pretrained_model_name, model_cls=model, config_kwargs= {"num_labels":num_labels} 
)

hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model) 

('roberta',
 transformers.models.roberta.configuration_roberta.RobertaConfig,
 transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast,
 transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification)

In [58]:
preprocessor = ClassificationPreprocessor(hf_tokenizer, label_mapping=labels)
dataset = preprocessor.process_hf_dataset(process_sample)
dataset

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', 'is_valid', 'label_name', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [60]:
dataset[2]

{'text': "This is an awesome classic monster flick from the 50's! I just love the look of the 50's in general like the cars and the music. Anyway, I love the way the blob looks. I love when the everyone is at the late night horror flick at the theater and the blob comes in and crashes the party. Another thing I love about it is that it takes place all in one night, just like Halloween II.<br /><br />When Steve and Jane are making out, they see a meteor fall from space. Inside the meteor is the blob. Whenever the blob consumes a person, it grows bigger and bigger. They try to convince the people of the town about the blobby monster, but no one believes them until later. Can anything stop this blobby creature? I highly recommend THE BLOB!!!",
 'label': 1,
 'is_valid': False,
 'label_name': 'pos',
 'input_ids': [0,
  152,
  16,
  41,
  6344,
  4187,
  13317,
  22154,
  31,
  5,
  654,
  18,
  328,
  38,
  95,
  657,
  5,
  356,
  9,
  5,
  654,
  18,
  11,
  937,
  101,
  5,
  1677,
  8,


In [61]:
#we can check what indexes the val set are at
val_idxs = [idx for idx, el in enumerate(datasetet) if el["is_valid"]==True]
min(val_idxs), max(val_idxs)

(1000, 1999)

In [62]:
blocks = (
    HF_TextBlock(
        hf_arch,
        hf_config,
        hf_tokenizer,
        hf_model,
        is_pretokenized=True,
        before_batch_kwargs={"labels": labels},
        tok_kwargs={"add_special_tokens": False},
    ),
    CategoryBlock,
)

datablock=DataBlock(blocks=blocks, get_x=ItemGetter("input_ids"), get_y=ItemGetter("label"), splitter=IndexSplitter(val_idxs))

In [63]:
dls = datablock.dataloaders(dataset, bs=4)
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


,text,target
0,"I'm gettin' sick of movies that sound entertaining in a one-line synopsis then end up being equal to what you'd find in the bottom center of a compost heap.<br /><br />Who knows: ""Witchery"" may have sounded interesting in a pitch to the studios, even with a ""big name cast"" (like Blair and Hasselhoff - wink-wink, nudge-nudge) and the effervescent likes of Hildegard Knef (I dunno, some woman...).<br /><br />But on film, it just falls apart faster than a papier-mache sculpture in a rainstorm. Seem",neg
1,"I was searching through Hollywood video last night with a friend trying to find a good-looking horror movie to watch over the New Year's weekend. As I was looking through the shelves, ""Severed"" spotted my eye, and I grabbed it off the shelf and it looked like it might be a decent B-grade horror movie. The cover looked fairly good. The plot sounded semi-interesting. So I rented it. What a mistake. Don't be fooled by the cover, which actually looks decent. I'm thinking that more money was spent o",neg


In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [72]:
torch.cuda.is_available()

False

In [69]:
mdl = HF_BaseModelWrapper(hf_model)
learn = Learner(
    dls,
    mdl,
    opt_func=partial(OptimWrapper, opt=torch.optim.Adam),
    loss_func=CrossEntropyLossFlat(),
    metrics=[accuracy],#its a list because we can look at an array of metrics if we wantback
    cbs=[HF_BaseModelCallback], #another list for an array of callback we could implement
    splitter=hf_splitter
)

In [70]:
learn.fit_one_cycle(2, lr_max=slice(1e-7,1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.461959,0.277662,0.892000,19:42


KeyboardInterrupt: 